# Importing Packages

In [137]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
import numpy as np

# Path

In [2]:
desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
industry_indices = os.path.join(desktop, "industry_indices")

# Config

In [3]:
limit = 3000

# Functions

In [71]:
def return_calculator(df):
    period_return = 1
    li = list(df)
    for day, _ in df.iteritems():
        idx = df.index.get_loc(day)
        elem = li[idx]
        if (math.isnan(elem)):
            elem = outlier_handler(elem)
            if elem == False:
                continue
        period_return = period_return * (1+elem)
    period_return = period_return - 1
    return period_return

def outlier_handler(ret):
    if (math.isnan(ret)):
        return False


In [4]:
download_financial_indexes(symbols="all", write_to_csv=True, base_path=industry_indices)

{'غذايي بجز قند':         value       date
 0       308.0 2008-12-05
 1       307.5 2008-12-06
 2       307.6 2008-12-07
 3       307.6 2008-12-08
 4       307.6 2008-12-10
 ...       ...        ...
 3392  49146.3 2022-12-21
 3393  50625.3 2022-12-24
 3394  51353.5 2022-12-25
 3395  51657.3 2022-12-26
 3396  53052.3 2022-12-28
 
 [3397 rows x 2 columns],
 'انبوه سازي':        value       date
 0      245.0 2008-12-05
 1      245.0 2008-12-06
 2      245.1 2008-12-07
 3      245.1 2008-12-08
 4      245.1 2008-12-10
 ...      ...        ...
 3392  7295.0 2022-12-21
 3393  7440.7 2022-12-24
 3394  7478.7 2022-12-25
 3395  7574.5 2022-12-26
 3396  7767.5 2022-12-28
 
 [3397 rows x 2 columns],
 'چند رشته اي ص':          value       date
 0        689.7 2008-12-05
 1        688.9 2008-12-06
 2        686.6 2008-12-07
 3        684.8 2008-12-08
 4        684.0 2008-12-10
 ...        ...        ...
 3392  204545.0 2022-12-21
 3393  210403.0 2022-12-24
 3394  212926.0 2022-12-25
 3395  213733.

In [5]:
list_of_files = os.listdir(industry_indices)
to_delete_indices = ["شاخص 30 شركت بزرگ.csv","شاخص آزاد شناور.csv","شاخص بازار اول.csv","شاخص بازار دوم.csv",
             "شاخص بازده نقدي قيمت.csv", "شاخص صنعت.csv", "شاخص قيمت (هم وزن).csv", "شاخص قيمت 50 شركت.csv",
             "شاخص قيمت(وزنيارزشي).csv", "شاخص50شركت فعالتر.csv", "شاخص كل.csv", "شاخص كل (هم وزن).csv"
             ]
defunct_sectors = ["مبلمان.csv", "ابزار پزشكي.csv" , "پيمانكاري.csv"]

In [6]:
sectors_li = [item for item in list_of_files if item not in to_delete_indices]
sectors_li = [item for item in sectors_li if item not in defunct_sectors]

In [7]:
for f in sectors_li:
    data = pd.read_csv(industry_indices+ "\\" + f)
    if len(data) < limit:
        sectors_li.remove(f)
    print(len(data),f)

1451 اداره بازارهاي مالي.csv
1274 اطلاعات و ارتباطات.csv
3397 انتشار و چاپ.csv
3397 بانكها.csv
3192 بيمه و بازنشسته.csv
1852 تامين آب، برق، گاز.csv
929 خرده فروشي به جز وسايل نقليه.csv
3397 دستگاههاي برقي.csv
3397 ذغال سنگ.csv
3397 راديويي.csv
3397 رايانه.csv
3398 زراعت.csv
3397 ساير معادن.csv
3397 سايرمالي.csv
3397 سرمايه گذاريها.csv
3397 سيمان.csv
3398 شيميايي.csv
3397 غذايي بجز قند.csv
3397 فراورده نفتي.csv
3397 فلزات اساسي.csv
3397 فني مهندسي.csv
3397 قند و شكر.csv
3397 كاشي و سراميك.csv
3397 كانه فلزي.csv
3397 كاني غيرفلزي.csv
3397 لاستيك.csv
3397 ماشين آلات.csv
3397 مالي.csv
3397 محصولات فلزي.csv
3397 محصولات كاغذ.csv
3396 محصولات چرمي.csv
3397 محصولات چوبي.csv
3397 منسوجات.csv
3398 مواد دارويي.csv
3397 وسايل ارتباطي.csv
3397 چند رشته اي ص.csv


In [8]:
len(sectors_li)

36

In [9]:
sectors_li[0]

'استخراج نفت جزكشف.csv'

In [130]:
data = pd.read_csv(industry_indices+ "\\" + sectors_li[0])
data['date'] = pd.to_datetime( data['date'],    format='%Y-%m-%d')
data["jdate"] = data["date"].jalali.to_jalali()
data["month"] = data["jdate"].jalali.month
data["year"] = data["jdate"].jalali.year
data["weekday"] = data["jdate"].jalali.weekday
#data.set_index('date' ,inplace=True)
data['value'] = data['value'].pct_change()
data['value'].values[data['value'].values > 0.05] = 0.05
data['value'].values[data['value'].values < -0.05] = -0.05
#weekly_data = data.resample("W").sum()

first_years_month = data["month"][0]
first_year = data["year"][0]
last_years_month = 12
last_year = 1396

empty_li_month = []
empty_li_year = []
for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_month += [i for i in range(first_years_month,13)]
            empty_li_year += [year] * (13-first_years_month)

        elif year == last_year:
            empty_li_month += [i for i in range(1,last_years_month+1)]
            empty_li_year += [year] * (last_years_month)

        else:
            empty_li_month += [i for i in range(1,13)]
            empty_li_year += [year] * 12

monthly_data = pd.DataFrame(columns=["return","year","month","return_1m","return_3m","return_6m"])
monthly_data["year"] = empty_li_year
monthly_data["month"] = empty_li_month

for year in range(first_year, last_year + 1):

    if year == first_year:
        for month in range(first_years_month, 13):
            idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
            ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["value"])
            monthly_data["return"][idx] = ret

    elif year == last_year:
        for month in range(1, last_years_month + 1):
            idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
            ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["value"])
            monthly_data["return"][idx] = ret

    else:
        for month in range(1, 13):
            idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
            ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["value"])
            monthly_data["return"][idx] = ret

monthly_data["return"] = monthly_data["return"] * 100
monthly_data["return_1m"] = monthly_data["return"].shift(1)
monthly_data["return_3m"] = monthly_data["return"].shift(3)
monthly_data["return_6m"] = monthly_data["return"].shift(6)
monthly_data = monthly_data.dropna().reset_index()
monthly_data = monthly_data.drop(['index'], axis=1)

C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\611742881.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, _ in df.iteritems():
C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\152148241.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_data["return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\152148241.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_data["return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\152148241.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [105]:
sectors_li[0]

'استخراج نفت جزكشف.csv'

In [131]:
index_data = pd.read_csv(industry_indices+ "\\" + "شاخص كل.csv")

index_data['date'] = pd.to_datetime( index_data['date'],    format='%Y-%m-%d')
index_data["jdate"] = index_data["date"].jalali.to_jalali()
index_data["month"] = index_data["jdate"].jalali.month
index_data["year"] = index_data["jdate"].jalali.year
index_data["weekday"] = index_data["jdate"].jalali.weekday
#data.set_index('date' ,inplace=True)
index_data['value'] = index_data['value'].pct_change()
index_data['value'].values[index_data['value'].values > 0.05] = 0.05
index_data['value'].values[index_data['value'].values < -0.05] = -0.05
#weekly_data = data.resample("W").sum()

first_years_month = index_data["month"][0]
first_year = index_data["year"][0]
last_years_month = 12
last_year = 1396

empty_li_month = []
empty_li_year = []
for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_month += [i for i in range(first_years_month,13)]
            empty_li_year += [year] * (13-first_years_month)

        elif year == last_year:
            empty_li_month += [i for i in range(1,last_years_month+1)]
            empty_li_year += [year] * (last_years_month)

        else:
            empty_li_month += [i for i in range(1,13)]
            empty_li_year += [year] * 12

index_monthly_data = pd.DataFrame(columns=["index_return","year","month","index_return_1m","index_return_3m","index_return_6m"])
index_monthly_data["year"] = empty_li_year
index_monthly_data["month"] = empty_li_month

for year in range(first_year, last_year + 1):

    if year == first_year:
        for month in range(first_years_month, 13):
            idx = index_monthly_data[(index_monthly_data.year == year) & (index_monthly_data.month == month)].index.item()
            ret = return_calculator(index_data[(index_data["month"] == month) & (index_data["year"] == year)]["value"])
            index_monthly_data["index_return"][idx] = ret

    elif year == last_year:
        for month in range(1, last_years_month + 1):
            idx = index_monthly_data[(index_monthly_data.year == year) & (index_monthly_data.month == month)].index.item()
            ret = return_calculator(index_data[(index_data["month"] == month) & (index_data["year"] == year)]["value"])
            index_monthly_data["index_return"][idx] = ret

    else:
        for month in range(1, 13):
            idx = index_monthly_data[(index_monthly_data.year == year) & (index_monthly_data.month == month)].index.item()
            ret = return_calculator(index_data[(index_data["month"] == month) & (index_data["year"] == year)]["value"])
            index_monthly_data["index_return"][idx] = ret

index_monthly_data["index_return"] = index_monthly_data["index_return"] * 100
index_monthly_data["index_return_1m"] = index_monthly_data["index_return"].shift(1)
index_monthly_data["index_return_3m"] = index_monthly_data["index_return"].shift(3)
index_monthly_data["index_return_6m"] = index_monthly_data["index_return"].shift(6)
index_monthly_data = index_monthly_data.dropna().reset_index()
index_monthly_data = index_monthly_data.drop(['index'], axis=1)

C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\611742881.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, _ in df.iteritems():
C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\3988837126.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_monthly_data["index_return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\3988837126.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_monthly_data["index_return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_5068\3988837126.py:51: SettingWithCopyWarning: 
A value is trying to be

In [113]:
index_data.to_csv("test.csv")

In [118]:
monthly_data

,index,return,year,month,return_1m,return_3m,return_6m
0,6,-457.368718,1388,11,675.105485,0.0,4116.504854
1,7,1048.638132,1388,12,-457.368718,-1882.805219,752.621838
2,8,681.458003,1389,1,1048.638132,675.105485,2301.204819
3,9,974.282888,1389,2,681.458003,-457.368718,0.0
4,10,-1744.028842,1389,3,974.282888,1048.638132,-1882.805219
...,...,...,...,...,...,...,...
93,99,-27.118644,1396,8,-438.634399,0.0,-769.440655
94,100,539.315658,1396,9,-27.118644,-1486.05732,-1009.45906
95,101,-1582.455386,1396,10,539.315658,-438.634399,-73.976656
96,102,-828.448584,1396,11,-1582.455386,-27.118644,0.0


In [117]:
index_monthly_data

,index,return,year,month,return_1m,return_3m,return_6m
0,6,286.196783,1388,3,675.585919,-489.57811,-423.617944
1,7,284.930442,1388,4,286.196783,583.317643,-439.986
2,8,1085.043372,1388,5,284.930442,675.585919,-107.234806
3,9,834.312344,1388,6,1085.043372,286.196783,-489.57811
4,10,606.717887,1388,7,834.312344,284.930442,583.317643
...,...,...,...,...,...,...,...
101,107,265.309285,1396,8,75.543097,210.931341,323.274602
102,108,758.550306,1396,9,265.309285,419.970767,-302.707097
103,109,346.419066,1396,10,758.550306,75.543097,245.706549
104,110,-67.670406,1396,11,346.419066,265.309285,210.931341


In [132]:
merged_data = monthly_data.merge(index_monthly_data, on=["month","year"])

In [133]:
merged_data["month_return"] = merged_data["return"] - merged_data["index_return"]
merged_data["1month_return"] = merged_data["return_1m"] - merged_data["index_return_1m"]
merged_data["3month_return"] = merged_data["return_3m"] - merged_data["index_return_3m"]
merged_data["6month_return"] = merged_data["return_6m"] - merged_data["index_return_6m"]

In [138]:
merged_data["1m_3m"] = 1/(1 + np.exp(-x))


TypeError: loop of ufunc does not support argument 0 of type float which has no callable exp method